In [1]:
import tensorflow as tf
import pathlib
import matplotlib.pyplot as plt
import numpy as np
from tensorflow import keras

2022-11-11 11:21:17.919672: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Download the kaggle dataset and put it in ./data/

[Dataset](https://www.kaggle.com/datasets/uraninjo/augmented-alzheimer-mri-dataset/download?datasetVersionNumber=1)

In [7]:
import zipfile
import os
import shutil

zip_location = './data/archive.zip'
extract_to = './data/'

with zipfile.ZipFile(zip_location, 'r') as zip_ref:
    zip_ref.extractall(extract_to)


os.remove("./data/archive.zip")
shutil.rmtree('./data/OriginalDataset') 


In [14]:
data = tf.keras.preprocessing.image_dataset_from_directory('./data/AugmentedAlzheimerDataset/')


Found 33984 files belonging to 4 classes.


In [23]:
# Resizing the images to 224x224
IMG_SIZE = 224
data = data.map(lambda x, y: (tf.image.resize(x, (IMG_SIZE, IMG_SIZE)), y))
# Shuffling the images
data = data.shuffle(buffer_size=1000)

In [24]:
import math
# Create the training/test split 80/20
train_ds = data.take(math.floor(0.8 * len(data)))
test_ds = data.skip(math.ceil(0.8 * len(data)))

# Print the sizes of the training/test sets
print("Training set size: ", len(train_ds))
print("Test set size: ", len(test_ds))

Training set size:  849
Test set size:  212


In [25]:
MobileNetv3Large = tf.keras.applications.MobileNetV3Small(
	input_shape=(224, 224, 3),
	include_top=False, 
	weights='imagenet')

# Transfer learning from MobileNetV3Large
MobileNetv3Large.trainable = False

# Create the model

model = tf.keras.Sequential([
	MobileNetv3Large,
	tf.keras.layers.GlobalAveragePooling2D(),
	tf.keras.layers.Dense(1, activation='sigmoid'),
	tf.keras.layers.Dropout(0.5),
])

# Compile the model

model.compile(
	optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
	loss=tf.keras.losses.BinaryCrossentropy(),
	metrics=['accuracy']
)

# Train the model
model.fit(train_ds, epochs=10)


4334752/4334752 [==============================] - 1s 0us/step
Epoch 1/10
117/849 [===>..........................] - ETA: 5:18 - loss: -0.4668 - accuracy: 0.1806

KeyboardInterrupt: 

In [ ]:
# Evaluate the model

model.evaluate(test_ds)
